<a href="https://colab.research.google.com/github/Sayed-Hossein-Hosseini/Dancing_with_Polynomial_Predictions/blob/master/Dancing_with_Polynomial_Predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Dancing with Polynomial Predictions**

## **Libraries**

In [1]:
import gdown
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

warnings.simplefilter(action='ignore', category=FutureWarning)

## **Dataset Description**

In [2]:
# Load the Excel file
file_path = 'Polynomial_Functions.xlsx'  # Change the path if needed
df = pd.read_excel(file_path, sheet_name='Sheet1')

# Display general information about the dataset
print("🔹 Dataset Information:")
print(df.info())

# Check for missing values in each column
print("\n🔹 Missing Values Per Column:")
print(df.isnull().sum())

# Show descriptive statistics
print("\n🔹 Descriptive Statistics:")
print(df.describe())

# Count unique values in each column
print("\n🔹 Number of Unique Values Per Column:")
print(df.nunique())


🔹 Dataset Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   x           10000 non-null  float64
 1   y           10000 non-null  float64
 2   z           10000 non-null  float64
 3   F(x, y, z)  10000 non-null  float64
dtypes: float64(4)
memory usage: 312.6 KB
None

🔹 Missing Values Per Column:
x             0
y             0
z             0
F(x, y, z)    0
dtype: int64

🔹 Descriptive Statistics:
                  x             y             z    F(x, y, z)
count  10000.000000  10000.000000  10000.000000  10000.000000
mean      -0.116809      0.090598      0.001008     43.803057
std        5.752603      5.785891      5.735475    587.941231
min       -9.999767     -9.996845     -9.999038  -3191.680304
25%       -5.073423     -4.921084     -4.925042   -225.322605
50%       -0.149428      0.117936      0.041362     15.622764
75%      

## **Train/Test Split**

In [3]:
# Separate features and target
X = df.drop('F(x, y, z)', axis=1)
y = df['F(x, y, z)']

# Split into training and test sets (e.g., 80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Display the sizes
print(f"Training set size: {X_train.shape[0]} rows")
print(f"Test set size: {X_test.shape[0]} rows")


Training set size: 8000 rows
Test set size: 2000 rows


## **Preprocessing**

### **Remove Outliers**

In [4]:
def remove_outliers_auto_xy_with_output(x, y):
    """
    Automatically detect and remove outliers from the training data (x, y) using the IQR method.
    Removes rows from both x and y simultaneously when an outlier is detected in either x or y.

    Parameters:
        x (pd.DataFrame): Input feature DataFrame (training data)
        y (pd.Series): Input target Series (labels)

    Returns:
        pd.DataFrame, pd.Series: Cleaned feature DataFrame and target Series with outliers removed
    """
    initial_shape = x.shape
    print(f"🔹 Initial dataset size: {initial_shape[0]} rows, {initial_shape[1]} columns (x), {y.shape[0]} rows (y)\n")

    # For numeric columns in x
    numeric_cols_x = x.select_dtypes(include='number').columns
    x_clean = x.copy()
    y_clean = y.copy()

    # Remove outliers from x
    for col in numeric_cols_x:
        Q1 = x[col].quantile(0.25)
        Q3 = x[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        before = x_clean.shape[0]
        mask = (x_clean[col] >= lower_bound) & (x_clean[col] <= upper_bound)

        # Apply the same mask to both x and y
        x_clean = x_clean[mask]
        y_clean = y_clean[mask]

        after = x_clean.shape[0]
        removed = before - after

        if removed > 0:
            print(f"🟠 Removed {removed} outliers from column '{col}' in x")
        else:
            print(f"✅ No outliers detected in column '{col}' in x")

    # For the target variable y
    Q1_y = y_clean.quantile(0.25)
    Q3_y = y_clean.quantile(0.75)
    IQR_y = Q3_y - Q1_y
    lower_bound_y = Q1_y - 1.5 * IQR_y
    upper_bound_y = Q3_y + 1.5 * IQR_y

    before_y = y_clean.shape[0]
    y_mask = (y_clean >= lower_bound_y) & (y_clean <= upper_bound_y)

    # Apply the same mask to both x and y
    x_clean = x_clean[y_mask]
    y_clean = y_clean[y_mask]

    after_y = x_clean.shape[0]
    removed_y = before_y - after_y

    if removed_y > 0:
        print(f"🟠 Removed {removed_y} outliers from target variable y")
    else:
        print(f"✅ No outliers detected in target variable y")

    print(f"\n✅ Final dataset size: {x_clean.shape[0]} rows (removed {initial_shape[0] - x_clean.shape[0]} total rows)")
    return x_clean, y_clean

# Example usage:
X_train_clean, y_train_clean = remove_outliers_auto_xy_with_output(X_train, y_train)


🔹 Initial dataset size: 8000 rows, 3 columns (x), 8000 rows (y)

✅ No outliers detected in column 'x' in x
✅ No outliers detected in column 'y' in x
✅ No outliers detected in column 'z' in x
🟠 Removed 603 outliers from target variable y

✅ Final dataset size: 7397 rows (removed 603 total rows)


### **Normalize Data**

In [5]:
def normalize_data(X_train, X_test, y_train, y_test):
    """
    This function normalizes the training and test features (X) and outputs (y).

    Parameters:
        X_train (numpy array): Training features
        X_test (numpy array): Test features
        y_train (numpy array): Training outputs
        y_test (numpy array): Test outputs

    Returns:
        X_train_normalized (numpy array): Normalized training features
        X_test_normalized (numpy array): Normalized test features
        y_train_normalized (numpy array): Normalized training outputs
        y_test_normalized (numpy array): Normalized test outputs
        feature_scaler (dict): Feature normalization parameters (mean, std)
        output_scaler (dict): Output normalization parameters (mean, std)
    """
    # Step 1: Normalize features
    feature_mean = np.mean(X_train, axis=0)  # Mean of each feature in X_train
    feature_std = np.std(X_train, axis=0)    # Standard deviation of each feature in X_train
    X_train_normalized = (X_train - feature_mean) / feature_std
    X_test_normalized = (X_test - feature_mean) / feature_std

    # Step 2: Normalize outputs
    output_mean = np.mean(y_train)  # Mean of y_train
    output_std = np.std(y_train)    # Standard deviation of y_train
    y_train_normalized = (y_train - output_mean) / output_std
    y_test_normalized = (y_test - output_mean) / output_std

    # Store normalization parameters for later use (e.g., denormalization)
    feature_scaler = {"mean": feature_mean, "std": feature_std}
    output_scaler = {"mean": output_mean, "std": output_std}

    return X_train_normalized, X_test_normalized, y_train_normalized, y_test_normalized, feature_scaler, output_scaler

# Example usage
# Assume you have the following datasets
# X_train = np.array([
#     [-2.5092, -2.52718, 4.59997],
#     [9.01429, -3.34176, -6.30976],
#     [4.63988, -6.47692, -3.06721],
#     [1.97137, 2.14533, 3.26561],
#     [-6.87963, -0.46752, -0.35821]
# ])

# X_test = np.array([
#     [-6.88011, 7.31402, 4.77142],
#     [-8.83833, -9.35781, 9.22416]
# ])

# y_train = np.array([100, 200, 300, 400, 500])
# y_test = np.array([600, 700])

# Normalize the data
X_train_norm, X_test_norm, y_train_norm, y_test_norm, feature_scaler, output_scaler = normalize_data(X_train_clean, X_test, y_train_clean, y_test)

# print(y_train_clean.head())
# Display normalized data
print("Normalized Training Features (X_train):")
print(X_train_norm.head())
print("\nNormalized Testing Features (X_test):")
print(X_test_norm.head())
print("\nNormalized Training Outputs (y_train):")
print(y_train_norm.head())
print("\nNormalized Testing Outputs (y_test):")
print(y_test_norm.head())

# Denormalize outputs (example)
def denormalize(data, scaler):
    return data * scaler["std"] + scaler["mean"]

# Example: Denormalize y_train_normalized
y_train_denormalized = denormalize(y_train_norm, output_scaler)
print("\nDenormalized Training Outputs (y_train):")
print(y_train_denormalized.head())

Normalized Training Features (X_train):
             x         y         z
9254  1.010234 -1.254771  1.617384
1561 -0.915031 -0.877572 -1.162636
1670 -0.396696 -0.584460 -0.077917
6087 -1.026385  1.302232  1.098537
5933  1.793400 -1.683188  0.390292

Normalized Testing Features (X_test):
             x         y         z
6252 -0.552270 -1.431263  1.687950
4684  1.027148  1.013973 -1.091141
1731  0.480483 -1.035976  1.761235
4742 -1.640797  1.072328 -1.331246
4521  0.311811  1.552574 -0.517810

Normalized Training Outputs (y_train):
9254    0.312019
1561   -1.838754
1670   -0.154669
6087    1.032377
5933   -1.259422
Name: F(x, y, z), dtype: float64

Normalized Testing Outputs (y_test):
6252   -1.631598
4684    0.934388
1731    0.044508
4742   -0.995126
4521    0.791898
Name: F(x, y, z), dtype: float64

Denormalized Training Outputs (y_train):
9254    154.530442
1561   -701.158409
1670    -31.142075
6087    441.126098
5933   -470.670206
Name: F(x, y, z), dtype: float64


## **Feature Engineering**

In [6]:
def feature_engineering(df):
    """
    This function takes a DataFrame with columns x, y, z as inputs
    and returns a feature matrix phi(x, y, z).

    The DataFrame must contain the columns 'x', 'y', and 'z'.
    """
    x = df['x']
    y = df['y']
    z = df['z']

    # Compute all the required terms
    phi = pd.DataFrame({
        'bias': 1,
        'x': x,
        'x2': x**2,
        'x3': x**3,
        'y': y,
        'y2': y**2,
        'y3': y**3,
        'z': z,
        'z2': z**2,
        'z3': z**3,
        'xy': x * y,
        'x2y': x**2 * y,
        'xy2': x * y**2,
        'xz': x * z,
        'x2z': x**2 * z,
        'xz2': x * z**2,
        'yz': y * z,
        'y2z': y**2 * z,
        'yz2': y * z**2,
        'xyz': x * y * z
    })

    return phi

# Example usage:
# Assuming X_train_norm and X_test_norm are DataFrames with columns 'x', 'y', 'z'
X_train_vector = feature_engineering(X_train_norm)
X_test_vector = feature_engineering(X_test_norm)

print("Feature Vector (Train):")
print(X_train_vector.head())
print("Feature Vector (Test):")
print(X_test_vector.head())

Feature Vector (Train):
      bias         x        x2        x3         y        y2        y3  \
9254     1  1.010234  1.020574  1.031019 -1.254771  1.574450 -1.975574   
1561     1 -0.915031  0.837282 -0.766139 -0.877572  0.770132 -0.675846   
1670     1 -0.396696  0.157367 -0.062427 -0.584460  0.341593 -0.199648   
6087     1 -1.026385  1.053467 -1.081262  1.302232  1.695809  2.208338   
5933     1  1.793400  3.216284  5.768084 -1.683188  2.833122 -4.768676   

             z        z2        z3        xy       x2y       xy2        xz  \
9254  1.617384  2.615930  4.230964 -1.267613 -1.280586  1.590564  1.633937   
1561 -1.162636  1.351724 -1.571563  0.803005 -0.734775 -0.704695  1.063849   
1670 -0.077917  0.006071 -0.000473  0.231853 -0.091975 -0.135509  0.030909   
6087  1.098537  1.206782  1.325695 -1.336592  1.371858 -1.740553 -1.127522   
5933  0.390292  0.152328  0.059452 -3.018629 -5.413611  5.080921  0.699950   

           x2z       xz2        yz       y2z       yz2       x